In [88]:
#import .env variables
from dotenv import dotenv_values
config = dotenv_values(".env")
config["USERNAME"]

'ssoundarajan@calmi2.org'

In [89]:
#imports
import boto3
import requests
import pandas as pd

In [90]:
def authenticate_user():
    PENNSIEVE_URL = "https://api.pennsieve.io"
    email = config["USERNAME"]
    password = config["PASSWORD"]

    r = requests.get(f"{PENNSIEVE_URL}/authentication/cognito-config")
    r.raise_for_status()
    cognito_app_client_id = r.json()["userPool"]["appClientId"]
    cognito_region = r.json()["userPool"]["region"]

    cognito_client = boto3.client(
    "cognito-idp",
    region_name=cognito_region,
    aws_access_key_id="",
    aws_secret_access_key="")
    
    login_response = cognito_client.initiate_auth(
    AuthFlow="USER_PASSWORD_AUTH",
    AuthParameters={"USERNAME": email, "PASSWORD": password},
    ClientId=cognito_app_client_id)
    
    api_key = login_response["AuthenticationResult"]["AccessToken"]
    
    r = requests.get(f"{PENNSIEVE_URL}/user", headers={"Authorization": f"Bearer {api_key}"})
    r.raise_for_status()
    print(r.json())
    return api_key 
    
api_key = authenticate_user()

{'id': 'N:user:65290a77-3d9e-4b43-9500-60a4a26ebc89', 'email': 'ssoundarajan@calmi2.org', 'firstName': 'Sanjay', 'lastName': 'Soundarajan', 'credential': 'Software Developer', 'color': '#474647', 'url': '', 'authyId': 0, 'isSuperAdmin': False, 'createdAt': '2020-12-07T23:37:58.087015Z', 'updatedAt': '2021-07-01T17:28:21.852925Z', 'preferredOrganization': 'N:organization:618e8dd9-f8d2-4dc4-9abb-c6aaab2e78a0', 'orcid': {'name': 'Sanjay Soundarajan', 'orcid': '0000-0003-2829-8032'}, 'pennsieveTermsOfService': {'version': '20210710000000'}, 'customTermsOfService': [], 'intId': 1079}


In [91]:
def get_published_datasets():
    url = "https://api.pennsieve.io/discover/search/records"
    querystring = {"model": "award"}
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    querystring = {"limit": response["totalCount"], "model": "award"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    list_of_datasets = response["records"].copy()
    return list_of_datasets
    
list_of_datasets = get_published_datasets()
list_of_datasets[0]

{'datasetId': 35,
 'version': 3,
 'model': 'award',
 'properties': {'description': 'PROJECT ABSTRACTThe peripheral and central elements of the respiratory control system are not “fixed,” but undergo sustained(neuroplastic) circuit reorganization to optimize function. This system can selectively utilize unique afferentmodalities and brainstem neural pathways to elicit episodic, coordinated airway protective behaviors (e.g.cough, laryngeal adduction). Neuroplasticity is induced and undermined by inflammation, transient afferentfeedback, or CNS injury. As a result, breathing responses and airway protective behaviors are altered in waysthat can be adaptive or maladaptive. Existing models of the brainstem network and sensory control systemregulating breathing and airway protection do not explain changes in responses caused by neuroplasticity insensory, central integrating and efferent motor elements of the control system. This knowledge gap concerningperipheral and central circuit-based pro

In [92]:
#change loop variant on prod
# for item in list_of_datasets[:1]:
#     url = f"https://api.pennsieve.io/discover/datasets/{item['datasetId']}/versions/{item['version']}/files"
#     querystring = {"path":"files/dataset_description.xlsx"}
#     headers = {"Accept": "application/json"}
#     response = requests.request("GET", url, headers=headers, params=querystring)
#     response = response.json()
#     item["datasetDescriptionPackageId"] = response["sourcePackageId"]
#     print(response)

In [99]:
#change loop variant on prod
for item in list_of_datasets[:1]:
    url = "https://api.pennsieve.io/zipit/discover"    
    payload = {"data": {
        "paths": ["files/dataset_description.xlsx"],
        "version": item['version'],
        "datasetId": item['datasetId']
    }}
    response = requests.request("POST", url, json=payload, headers=headers)
    local_filename = "dataset_description.xlsx"
    totalbits = 0
    if response.status_code == 200:
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    totalbits += 1024
#                     print("Downloaded",totalbits*1025,"KB...")
                    f.write(chunk)
        xl = pd.ExcelFile("dataset_description.xlsx")
        df = xl.parse("Sheet1")

In [100]:
xl = pd.ExcelFile("dataset_description.xlsx")
df = xl.parse("Sheet1")
df

,Metadata element,Description,Example,Value,Value 2,Value 3,Additional Values…,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Name,Descriptive title for the data set. Equivalent...,NaN,Brainstem neuron recordings - Morris Lab USF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Description,Brief description of the study and the data se...,NaN,Simultaneously-recorded cat brainstem neuron e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Keywords,A set of 3-5 keywords other than the above tha...,NaN,swallow,superior laryngeal nerve,electrical stimulation,respiration,neural circuits,phrenic,hypoglossal,blood pressure,tracheal pressure,expired CO2,NaN,NaN
3,Contributors,Name of any contributors to the dataset. Thes...,"Last, First Middle","Morris, Kendall F.","Horton, Kofi-Kermit","Segers, Lauren S","Nuding, Sarah C.","Gestreau, Christian","Alencar, Pierina A.","Shuman, H. Dale","O'Connor, Russell","Lindsey, Bruce G.","Bolser, Donald C.","Davenport, Paul W.","Pitts, Teresa"
4,Contributor ORCID ID,"ORCID ID. If you don't have an ORCID, we sugge...",NaN,https://orcid.org/0000-0001-6582-1203,https://orcid.org/0000-0002-1238-5257,https://orcid.org/0000-0003-1058-1569,NaN,https://orcid.org/0000-0001-5069-482X,NaN,NaN,NaN,NaN,https://orcid.org/0000-0002-4577-8954,NaN,https://orcid.org/0000-0001-9448-2175\t
5,Contributor Affiliation,Institutional affiliation for contributors,NaN,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Molecular Pharmacology and Physiology\nMorsani...,Department of Molecular Pharmacology and Physi...,Department of Physiological Sciences\nSchool o...,Department of Physiological Sciences\nSchool o...,"University of Louisville\nLouisville, KY"
6,Contributor Role,"Contributor role, e.g., PrincipleInvestigator,...",Data Collector,"Creator, PrincipleInvestigator\n",Researcher,Researcher,Researcher,Researcher,Researcher,DataManager,DataManager,Researcher,Researcher,Researcher,Researcher
7,Is Contact Person,Yes or No if the contributor is a contact pers...,Yes,Yes,No,Yes,No,No,No,No,No,No,Yes,No,No
8,Acknowledgements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Funding,Funding sources (e.g. NIH OT3-OD025349),NaN,OT2OD023854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
